# Mapping Spruce Bark Beetle Futures in Alaska's Central Interior Parks

This notebook demonstrates how to query the SNAP Data API for Alaska climate division polygons, Alaska protected area polygons, and modeled climate protection from spruce bark beetles. In this example, we use ARDAC helper functions from the ```ardac_utils.py``` module to quickly fetch and summarize these datasets.

Beetle protection categories (none, minimal, and high) for a number of climate and emissions scenarios are then presented with an interactive map, where clicking on a protected area polygon displays the data values.

In [1]:
from ardac_utils import *
from ipyleaflet import Map, GeoData, GeoJSON, Popup, basemaps
from ipywidgets import Layout, interactive, fixed
import matplotlib.pyplot as plt

### Fetch polygons using the SNAP Data API

Use the ```get_area_gdf_by_category()``` helper function to fetch climate division and protected area GeoJSON polygons from the SNAP Data API and convert them to `GeoDataFrame` objects. 

The category string that we pass to these functions is identical to the endpoint URL found in the [documentation](https://earthmaps.io/boundary/) for the physical and administrative boundary datasets. For example, if one would use ```earthmaps.io/places/corporations``` to get Alaska Native Corporation data, we can pass the last part of this URL (```"corporations"```) as input to the helper function.

Passing `3338` as the second argument tells the function that we would like the polygons to be returned in the Alaska Albers projection (EPSG:3338).

In [2]:
region_gdf = get_area_gdf_by_category("climate_divisions", 3338)
parks_gdf = get_area_gdf_by_category("protected_areas", 3338)

### Subset polygons

In the climate regions `GeoDataFrame`, the three regions of the Alaska interior are CD3, CD5, and CD6 (Central, Northeast, and Southeast Interior, respectively). We will just use the Central Interior in this demonstration, so first we subset the original to a new `GeoDataFrame`.

Then we use a spatial join (`sjoin()`) to find the protected area polygons that have at least some portion contained within the Central Interior polygon. We keep the protected area geometry by specifying a "right" join, and then drop any `NA` rows to remove protected areas that are not in the Central Interior.

In [3]:
interior_region = region_gdf[region_gdf['id']=='CD3'].copy()
interior_parks_gdf = interior_region.sjoin(parks_gdf, how='right', predicate='contains').dropna()
interior_parks_gdf.head()

,index_left,id_left,name_left,id_right,name_right,the_geom
0,0.0,CD3,Central Interior,BLM1,Arms Lake Research Natural Area,"MULTIPOLYGON (((80238.500 1656842.223, 80249.5..."
0,0.0,CD3,Central Interior,BLM5,Dulbi-Kaiyuh Mountains Area Of Critical Enviro...,"MULTIPOLYGON (((-122732.244 1630434.874, -1225..."
0,0.0,CD3,Central Interior,BLM8,Galena Mountain Watershed Area Of Critical Env...,"MULTIPOLYGON (((-95669.042 1667219.872, -94580..."
0,0.0,CD3,Central Interior,BLM9,George River Area Of Critical Environmental Co...,"MULTIPOLYGON (((-161282.463 1317014.836, -1614..."
0,0.0,CD3,Central Interior,BLM12,Hogatza Area Of Critical Environmental Concern,"MULTIPOLYGON (((-68721.821 1801163.765, -69421..."


### Fetch beetles data using the SNAP Data API

Note that since both original `GeoDataFrame`s had `index` and `id` columns, we now have `left_` and `right_` suffixes attached to some of the column names. Let's clean up our `GeoDataFrame`, and then use it to fetch spruce bark beetle data from the SNAP Data API. 

Here we use another ARDAC helper function, `get_data_for_gdf_polygons()`, which uses our input polygons to request zonal statistics from API endpoints. We need a clean `id` column in the `GeoDataFrame` that we pass to the helper function. The function appends each `id` to an API request, and returns zonal statistics for the corresponding polygon. **This function will only work with polygons and polygon IDs recognized by the SNAP Data API!** You cannot just use any polygon ID - it must be one of the IDs referenced in the API documentation [here](https://earthmaps.io/boundary/#:~:text=Service%20endpoints-,Available%20places,-Query%20a%20list) under "available places". 

**Note:** Not all data endpoints in the API offer zonal statistics. If you request data from an endpoint that does not have zonal stats, you will receive an error message which lists all possible choices.

**Also note:** If a polygon is smaller in area than the raster cells you are querying, you will receive an error because the polygon does not completely contain at least one raster cell. In these cases, it is better to use a latitude / longitude point query instead of a polygon. You will notice below that several small protected areas do not return any beetles data. This could be because the beetles dataset does not cover these areas, but in this case it is more likely that these polygons are simply smaller than an individual raster cell.

In [4]:
interior_parks_gdf.drop(columns=['index_left', 'id_left', 'name_left'], inplace=True)
interior_parks_gdf.rename(columns={'id_right':'id', 'name_right':'name'}, inplace=True)

interior_parks_beetles_data = get_data_for_gdf_polygons(interior_parks_gdf, "beetles")

Bad request, no beetles data for BLM1: Arms Lake Research Natural Area...trying next polygon...
Bad request, no beetles data for BLM8: Galena Mountain Watershed Area Of Critical Environmental Concern...trying next polygon...
Bad request, no beetles data for BLM12: Hogatza Area Of Critical Environmental Concern...trying next polygon...
Bad request, no beetles data for BLM20: Mcquesten Creek Research Natural Area...trying next polygon...
Bad request, no beetles data for BLM24: Nugget Creek Area Of Critical Environmental Concern...trying next polygon...
Bad request, no beetles data for BLM25: Nulato Hills Threatened And Endangered Area...trying next polygon...
Bad request, no beetles data for BLM27: Poss Mountain Area Of Critical Environmental Concern...trying next polygon...
Bad request, no beetles data for BLM28: Redlands Lake Research Natural Area...trying next polygon...
Bad request, no beetles data for BLM31: Snowden Mountain Area Of Critical Environmental Concern...trying next polyg

In [18]:
interior_parks_beetles_data

,id,name,era,model,scenario,snowpack,climate protection,percent high protection,percent minimal protection,percent no protection
0,BLM5,Dulbi-Kaiyuh Mountains Area Of Critical Enviro...,1988-2017,Daymet,Historical,low,high,100.0,0.0,0.0
1,BLM5,Dulbi-Kaiyuh Mountains Area Of Critical Enviro...,1988-2017,Daymet,Historical,medium,high,100.0,0.0,0.0
2,BLM5,Dulbi-Kaiyuh Mountains Area Of Critical Enviro...,2010-2039,NCAR-CCSM4,rcp45,low,high,100.0,0.0,0.0
3,BLM5,Dulbi-Kaiyuh Mountains Area Of Critical Enviro...,2010-2039,NCAR-CCSM4,rcp45,medium,high,100.0,0.0,0.0
4,BLM5,Dulbi-Kaiyuh Mountains Area Of Critical Enviro...,2010-2039,NCAR-CCSM4,rcp85,low,high,100.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
45,BLM26,Oskawalik River Area Of Critical Environmental...,2070-2099,HadGEM2-ES,rcp85,medium,high,50.0,50.0,0.0
46,BLM26,Oskawalik River Area Of Critical Environmental...,2070-2099,MRI-CGCM3,rcp45,low,high,100.0,0.0,0.0
47,BLM26,Oskawalik River Area Of Critical Environmental...,2070-2099,MRI-CGCM3,rcp45,medium,minimal,0.0,100.0,0.0
48,BLM26,Oskawalik River Area Of Critical Environmental...,2070-2099,MRI-CGCM3,rcp85,low,high,100.0,0.0,0.0


### Plot widgets on an interactive map

Frist create a plotting function that will take the protected area name (`loc`), the emissions scenario (`Scenario`), the snowpack level (`Snowpack`), and a dataframe (`df`) as arguments. This function will first subset the dataframe input using the first 3 arguments, then create a bar plot that counts the climate protection results in each category (ie, "none", "minimal", or "high") for each era. 

Note that we are not specifying model here; instead, we count the responses over all models. In this way, the different models are "voting" their result under the given scenario and snowpack level. This is an appropriate method of aggregating the beetle data, since the model output is categorical in nature. We cannot take a mean, but we can count the result categories across models and see where climate protection results are unanimous, leaning in one direction, or mixed.

In [71]:
def plot(loc, Scenario, Snowpack, df):

    plot_df = df[(df['scenario']==Scenario) & 
                 (df['snowpack']==Snowpack) &
                 (df['name']==loc)]

    fig, ax = plt.subplots(layout='constrained')

    p = plot_df.groupby(['climate protection'])['era'].value_counts().unstack(0).plot.bar(ax=ax)

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_xlabel(None)
    ax.set_ylabel("Vote Count")
    ax.set_yticks([1,2,3,4])
    ax.set_yticklabels(['1', '2', '3', '4'])
    ax.set_title('Climate Protection from Spruce Beetles\nFour Model Vote by Era')
    ax.legend(loc='upper right')

    return p


Now we can prep the data for mapping. Since there were some protected area polygons that did not return beetles data, we first exclude those from the map by subsetting to include only those polygons with names in the `interior_parks_beetles_data` `GeoDataFrame`. Then we extract the centroid lat/lon from the entire `GeoDataFrame` to use in centering the map view, and convert the `GeoDataFrame` to GeoJSON.

In [ ]:
#subset polygons
map_gdf = interior_parks_gdf[interior_parks_gdf['name'].isin(interior_parks_beetles_data.name.unique().tolist())].copy()
#find center of the GeoDataFrame to center map view
center = (map_gdf.dissolve().centroid.to_crs(4326).y[0], map_gdf.dissolve().centroid.to_crs(4326).x[0])
#convert geodataframe to GeoJSON object
parks_poly = GeoData(geo_dataframe = map_gdf.to_crs(4326))

Finally, let's create the map object. After setting some basic formatting options, we initialize an interactive widget that will contain two drop-down menus with choices for emissions scenario and snowpack levels. The widget also has some fixed values for the location (derived from the GeoJSON object above) and the beetles `DataFrame`. 



ETC ETC ETC

In [ ]:
map = Map(
    basemap=basemaps.Esri.WorldTopoMap,
    center=center,
    zoom=6,
    layout=Layout(width='90%', height='500px'))

for feature in parks_poly.data['features']:
    geojson = GeoJSON(data=feature, 
            style={'color': 'black', 'fillColor': 'blue', 'opacity':0.1, 'weight':5, 'fillOpacity':0.6},
            hover_style={'fillColor': 'lightblue' , 'fillOpacity': 0.2},
            )
    name = feature['properties']['name']

    widget_popup = interactive(plot, 
                               Scenario=['rcp45', 'rcp85'],
                               Snowpack=['low', 'medium'], 
                               loc=fixed(name), 
                               df=fixed(interior_parks_beetles_data))

    # Popup with the interactive widget
    popup = Popup(
        location=center,
        child=widget_popup,
        close_button=True,
        auto_close=True,
        minWidth=700
        )

    geojson.popup = popup
    map.add_layer(geojson)

display(map)